In [27]:
import os
import json
import math
import numpy as np

def get_transformation_matrix(fr5_dh_parameters, theta):
    a = fr5_dh_parameters['a']
    d = fr5_dh_parameters['d']
    alpha = math.radians(fr5_dh_parameters['alpha'])
    theta = math.radians(fr5_dh_parameters['theta'] + theta)
    
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

angle_path = '/home/najo/NAS/DIP/datasets/FR5_model/angle'
save_path = '/home/najo/NAS/DIP/datasets/FR5_model/joint'
angle_file = [f for f in os.listdir(angle_path) if f.endswith('.json')]

fr5_dh_parameters = [
    {'alpha': 90, 'a': 0, 'd': 0.152, 'theta': 0},
    {'alpha': 0, 'a': -0.425, 'd': 0, 'theta': 0},
    {'alpha': 0, 'a': -0.395, 'd': 0, 'theta': 0},
    {'alpha': 90, 'a': 0, 'd': 0.102, 'theta': 0},
    {'alpha': -90, 'a': 0, 'd': 0.102, 'theta': 0},
    {'alpha': 0, 'a': 0, 'd': 0.100, 'theta': 0}
]

for i in range(len(angle_file)):
    with open(os.path.join(angle_path, angle_file[i]), 'r') as file:
                data = json.load(file)
                
    base_coordinate = np.array([[0], 
                        [0], 
                        [0], 
                        [1]])

    T_1 = get_transformation_matrix(fr5_dh_parameters[0], data[0])
    T_2 = get_transformation_matrix(fr5_dh_parameters[1], data[1])
    T_3 = get_transformation_matrix(fr5_dh_parameters[2], data[2])
    T_4 = get_transformation_matrix(fr5_dh_parameters[3], data[3])
    T_5 = get_transformation_matrix(fr5_dh_parameters[4], data[4])
    T_6 = get_transformation_matrix(fr5_dh_parameters[5], data[5])

    JOINT_1 = T_1 @ base_coordinate
    JOINT_2 = T_1 @ T_2 @ base_coordinate
    JOINT_3 = T_1 @ T_2 @ T_3 @ base_coordinate
    JOINT_4 = T_1 @ T_2 @ T_3 @ T_4 @ base_coordinate
    JOINT_5 = T_1 @ T_2 @ T_3 @ T_4 @ T_5 @ base_coordinate
    JOINT_6 = T_1 @ T_2 @ T_3 @ T_4 @ T_5 @ T_6 @ base_coordinate

    # json 파일 저장하기
    joint_data = [
        JOINT_1[:3, 0].tolist(),
        JOINT_2[:3, 0].tolist(),
        JOINT_3[:3, 0].tolist(),
        JOINT_4[:3, 0].tolist(),
        JOINT_5[:3, 0].tolist(),
        JOINT_6[:3, 0].tolist()
    ]

    with open(os.path.join(save_path, angle_file[i]), 'w') as file:
        json.dump(joint_data, file, indent=4)
print("DONE")

DONE
